In [1]:
import os
import sys
from datetime import datetime
from IPython.display import HTML

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

WIDTH = 1100
pd.options.display.float_format = 'R$ {:,.2f}'.format

caminho_commons = os.path.join('../..', '..', 'ajna_docs', 'commons')
caminho_virasana = os.path.join('../..', '..', 'ajna_docs', 'virasana')
sys.path.append(caminho_commons)
sys.path.append('../..')
sys.path.append(caminho_virasana)

from ajna_commons.flask.conf import DATABASE, MONGODB_URI

from bhadrasana.models import engine


def AnoMes(row):
    return str(row.Ano) + str(row['Mês']).zfill(2)


2022-05-16 09:01:15,611 ajna         INFO     Configuração de log efetuada


Fazendo log de erros e alertas no arquivo  ../..\..\ajna_docs\commons\ajna_commons\flask\error.log
Fazendo log de atividade no arquivo  ../..\..\ajna_docs\commons\ajna_commons\flask\access.log


# Colocar os códigos embaixo

In [2]:
#  4 é fase arquivada sem resultado, eliminar!!!
SQL_OUTLET = '''SELECT r.nome as Recinto, ficha.id as 'Ficha', rvf.id as 'RVF', rvf.datahora 'Data de Abertura',
     ficha.numeroCEmercante, rvf.numerolote as Contêiner, ficha.fase
     FROM ovr_ovrs ficha
     INNER JOIN ovr_verificacoesfisicas rvf ON  rvf.ovr_id = ficha.id
     inner join ovr_flags_ovr flags on flags.rvf_id = ficha.id
     INNER JOIN ovr_recintos r ON r.id = ficha.recinto_id
     where flags.flag_id = 9
     ORDER BY r.nome, rvf.datahora'''

df_pendente_outlet = pd.read_sql(SQL_OUTLET, engine)
today_ = datetime.today()
def calcula_dias(x):
    try:
        return int((today_ - x['Data de Abertura']).days)
    except:
        return 0
            

df_pendente_outlet['Dias sem conclusão'] = \
    df_pendente_outlet.apply(calcula_dias, axis=1)
lista_ovr = ', '.join([str(i) for i in df_pendente_outlet.Ficha.values])

SQL_EVENTOS = \
    '''SELECT ovr_id as Ficha, tipoevento_id, t.nome, e.create_date, motivo FROM ovr_eventos e
    inner join ovr_tiposevento t on e.tipoevento_id = t.id  where ovr_id in (%s)
    ORDER BY ovr_id;'''

df_eventos = pd.read_sql(SQL_EVENTOS % lista_ovr, engine)


# Tipos de Evento : 36 Em análise da Fiscalização
#                    7 Aguardando quantificação do Recinto
#                   21 Termo de Guarda informado
#                   8 Recebimento de quantificação do Recinto
df_pendente_outlet = df_pendente_outlet.drop_duplicates(subset=['Ficha'])
df_pendente_outlet = df_pendente_outlet.set_index('Ficha')
df_pendente_outlet = df_pendente_outlet.join(
    df_eventos[df_eventos.tipoevento_id == 36].drop_duplicates(
        ('Ficha', 'tipoevento_id'))[['Ficha', 'motivo']].set_index('Ficha')
)

In [3]:
df_pendente_outlet = df_pendente_outlet.rename(columns={'motivo': 'Parecer'}).fillna('--')

In [4]:
df_pendente_outlet.replace({'SANTOS/BRASIL SA OPERADOR PORTUARIO': 'SANTOS BRASIL PARTICIPACOES LTDA' }, inplace=True)

In [5]:
tipos_eventos = {36: 'Análise',
                 7: 'Saneamento solicitado',
                 8: 'Saneamento recebido',
                 21: 'TG recebido'}
for tipoevento_id, descricao in tipos_eventos.items():
    df_pendente_outlet = df_pendente_outlet.join(
        df_eventos[df_eventos.tipoevento_id == tipoevento_id].drop_duplicates(
            ('Ficha', 'tipoevento_id'))[['Ficha', 'create_date']].set_index('Ficha')
    )
    df_pendente_outlet = df_pendente_outlet.rename(columns={'create_date': descricao}).fillna('Sem evento')

In [6]:
recintos = df_pendente_outlet['Recinto'].unique()
df_pendente_outlet['Recinto'].value_counts()

EUDMARCO - PORTO SECO                                 23
SANTOS BRASIL PARTICIPACOES LTDA                      18
BANDEIRANTES DEICMAR - IPA OUTEIRINHOS                12
BANDEIRANTES DEICMAR - CLIA ALEMOA                     7
SANTOS BRASIL LOGISTICA S/A -CLIA SANTOS               3
TRANSBRASA IPA                                         3
Brasil Terminal Portuario                              1
EMBRAPORT - EMPRESA BRASILEIRA DE TERMINAIS PORTUA     1
SANTOS BRASIL LOGISTICA S/A -CLIA GUARUJA              1
Name: Recinto, dtype: int64

In [7]:
def duracao(x):
    try:
        return (x['Saneamento recebido'] - x['Data de Abertura']).days
    
    except:
        try:
            return (x['TG recebido'] - x['Data de Abertura']).days
        except:
            return 0
df_pendente_outlet['Duração'] = df_pendente_outlet.apply(duracao, axis=1)

In [8]:
len(df_pendente_outlet[df_pendente_outlet['Recinto'] == 'BANDEIRANTES DEICMAR - CLIA ALEMOA'])

7

In [9]:
def mostra_recinto(recinto, filtrar_concluidos=False):
    print(recinto, len(df_pendente_outlet[df_pendente_outlet['Recinto'] == recinto]))
    df_pendentes = df_pendente_outlet.reset_index()
    df_pendentes = df_pendentes[['Ficha', 'Recinto', 'Data de Abertura', 'numeroCEmercante', 'Dias sem conclusão',
                                 'Parecer', 'Contêiner', 'Saneamento recebido', 'TG recebido', 'fase', 'Duração']]
    df_pendentes.sort_values(['Ficha'], inplace=True)
    if filtrar_concluidos:
        df_pendentes=df_pendentes[df_pendentes['fase'] != 3]
        df_pendentes=df_pendentes[df_pendentes['fase'] != 4]
        df_pendentes=df_pendentes[df_pendentes['TG recebido'] == 'Sem evento']
    return df_pendentes[df_pendentes['Recinto'] == recinto]

In [10]:
mostra_recinto(recintos[0])

BANDEIRANTES DEICMAR - CLIA ALEMOA 7


,Ficha,Recinto,Data de Abertura,numeroCEmercante,Dias sem conclusão,Parecer,Contêiner,Saneamento recebido,TG recebido,fase,Duração
2,2225,BANDEIRANTES DEICMAR - CLIA ALEMOA,2021-07-19 10:10:00,152105110560240,300,desbloqueio usados,CLHU8513070,2022-03-15 16:59:56,Sem evento,1,239
0,2268,BANDEIRANTES DEICMAR - CLIA ALEMOA,2021-06-29 11:51:00,152105130137807,320,--,MRSU4106682,Sem evento,Sem evento,1,0
1,2269,BANDEIRANTES DEICMAR - CLIA ALEMOA,2021-07-06 12:02:00,152105132206466,313,Desbloqueio dos usados,TCNU1731746,2022-03-15 16:48:36,Sem evento,1,252
3,2398,BANDEIRANTES DEICMAR - CLIA ALEMOA,2021-07-27 14:55:00,152105130139770,292,PEC - encaminhar para apreensão total,TGBU6766220,Sem evento,Sem evento,2,0
4,2517,BANDEIRANTES DEICMAR - CLIA ALEMOA,2021-08-20 16:25:00,152105187235816,268,Separar itens novos encontrados: bens novos nã...,MRKU3234520,Sem evento,Sem evento,2,0
6,2518,BANDEIRANTES DEICMAR - CLIA ALEMOA,2021-08-30 16:27:00,152105180651008,258,--,UACU6038187,2022-04-19 15:39:21,Sem evento,1,231
5,2534,BANDEIRANTES DEICMAR - CLIA ALEMOA,2021-08-27 10:00:00,152105154382151,261,Encaminhar para apreensão total - Abrir PEC,MRSU3000994,Sem evento,Sem evento,2,0


In [11]:
mostra_recinto(recintos[0], True).drop(columns=['Parecer', 'fase']).style.hide_index()

BANDEIRANTES DEICMAR - CLIA ALEMOA 7


Ficha,Recinto,Data de Abertura,numeroCEmercante,Dias sem conclusão,Contêiner,Saneamento recebido,TG recebido,Duração
2225,BANDEIRANTES DEICMAR - CLIA ALEMOA,2021-07-19 10:10:00,152105110560240,300,CLHU8513070,2022-03-15 16:59:56,Sem evento,239
2268,BANDEIRANTES DEICMAR - CLIA ALEMOA,2021-06-29 11:51:00,152105130137807,320,MRSU4106682,Sem evento,Sem evento,0
2269,BANDEIRANTES DEICMAR - CLIA ALEMOA,2021-07-06 12:02:00,152105132206466,313,TCNU1731746,2022-03-15 16:48:36,Sem evento,252
2398,BANDEIRANTES DEICMAR - CLIA ALEMOA,2021-07-27 14:55:00,152105130139770,292,TGBU6766220,Sem evento,Sem evento,0
2517,BANDEIRANTES DEICMAR - CLIA ALEMOA,2021-08-20 16:25:00,152105187235816,268,MRKU3234520,Sem evento,Sem evento,0
2518,BANDEIRANTES DEICMAR - CLIA ALEMOA,2021-08-30 16:27:00,152105180651008,258,UACU6038187,2022-04-19 15:39:21,Sem evento,231
2534,BANDEIRANTES DEICMAR - CLIA ALEMOA,2021-08-27 10:00:00,152105154382151,261,MRSU3000994,Sem evento,Sem evento,0


In [12]:
mostra_recinto(recintos[1])

BANDEIRANTES DEICMAR - IPA OUTEIRINHOS 12


,Ficha,Recinto,Data de Abertura,numeroCEmercante,Dias sem conclusão,Parecer,Contêiner,Saneamento recebido,TG recebido,fase,Duração
12,2227,BANDEIRANTES DEICMAR - IPA OUTEIRINHOS,2021-07-20 09:00:00,152105110537001,300,desbloqueio usados,MEDU7969615,Sem evento,Sem evento,1,0
8,2241,BANDEIRANTES DEICMAR - IPA OUTEIRINHOS,2021-06-24 17:05:00,152105129681590,325,desbloqueio usados,MEDU4356862,2022-05-16 08:48:51,Sem evento,1,325
7,2251,BANDEIRANTES DEICMAR - IPA OUTEIRINHOS,2021-06-21 16:55:00,152105125111676,328,--,,Sem evento,2021-11-26 12:37:32,3,157
10,2252,BANDEIRANTES DEICMAR - IPA OUTEIRINHOS,2021-07-12 08:30:00,152105137703018,308,--,HLBU1628819,Sem evento,Sem evento,2,0
9,2253,BANDEIRANTES DEICMAR - IPA OUTEIRINHOS,2021-07-02 18:23:00,152105142310304,317,Perdimento por falsa declaração de conteúdo e ...,AMFU8799788,Sem evento,Sem evento,2,0
16,2254,BANDEIRANTES DEICMAR - IPA OUTEIRINHOS,2021-08-05 14:58:00,152105137703107,283,Encaminhar para apreensão total - PEC,HLXU8324986,Sem evento,2021-12-15 14:25:25,1,131
11,2316,BANDEIRANTES DEICMAR - IPA OUTEIRINHOS,2021-07-15 14:38:00,152105122600919,304,Encaminhar para apreensão total por interposiç...,MRKU5233924,Sem evento,Sem evento,1,0
18,2318,BANDEIRANTES DEICMAR - IPA OUTEIRINHOS,2021-09-09 09:00:00,152105118983868,249,--,MSKU9981464,Sem evento,Sem evento,2,0
14,2319,BANDEIRANTES DEICMAR - IPA OUTEIRINHOS,2021-07-22 08:30:00,152105154382232,298,Encaminhar para apreensão total - PEC enviada,TRHU7614653,Sem evento,Sem evento,2,0
15,2356,BANDEIRANTES DEICMAR - IPA OUTEIRINHOS,2021-08-02 15:06:00,152105151830390,286,Encaminhar para apreensão total - fazer PEC,FFAU2140262,Sem evento,Sem evento,1,0


In [13]:
mostra_recinto(recintos[1], True).drop(columns=['Parecer', 'fase']).style.hide_index()

BANDEIRANTES DEICMAR - IPA OUTEIRINHOS 12


Ficha,Recinto,Data de Abertura,numeroCEmercante,Dias sem conclusão,Contêiner,Saneamento recebido,TG recebido,Duração
2227,BANDEIRANTES DEICMAR - IPA OUTEIRINHOS,2021-07-20 09:00:00,152105110537001,300,MEDU7969615,Sem evento,Sem evento,0
2241,BANDEIRANTES DEICMAR - IPA OUTEIRINHOS,2021-06-24 17:05:00,152105129681590,325,MEDU4356862,2022-05-16 08:48:51,Sem evento,325
2252,BANDEIRANTES DEICMAR - IPA OUTEIRINHOS,2021-07-12 08:30:00,152105137703018,308,HLBU1628819,Sem evento,Sem evento,0
2253,BANDEIRANTES DEICMAR - IPA OUTEIRINHOS,2021-07-02 18:23:00,152105142310304,317,AMFU8799788,Sem evento,Sem evento,0
2316,BANDEIRANTES DEICMAR - IPA OUTEIRINHOS,2021-07-15 14:38:00,152105122600919,304,MRKU5233924,Sem evento,Sem evento,0
2318,BANDEIRANTES DEICMAR - IPA OUTEIRINHOS,2021-09-09 09:00:00,152105118983868,249,MSKU9981464,Sem evento,Sem evento,0
2319,BANDEIRANTES DEICMAR - IPA OUTEIRINHOS,2021-07-22 08:30:00,152105154382232,298,TRHU7614653,Sem evento,Sem evento,0
2356,BANDEIRANTES DEICMAR - IPA OUTEIRINHOS,2021-08-02 15:06:00,152105151830390,286,FFAU2140262,Sem evento,Sem evento,0
2399,BANDEIRANTES DEICMAR - IPA OUTEIRINHOS,2021-08-09 15:00:00,152105154381503,279,MSKU1453373,Sem evento,Sem evento,0


In [14]:
mostra_recinto(recintos[7], True).drop(columns=['Parecer', 'fase']).style.hide_index()

SANTOS BRASIL PARTICIPACOES LTDA 18


Ficha,Recinto,Data de Abertura,numeroCEmercante,Dias sem conclusão,Contêiner,Saneamento recebido,TG recebido,Duração
2242,SANTOS BRASIL PARTICIPACOES LTDA,2021-07-06 09:00:00,152105130139699,314,CAIU8199797,2022-04-19 15:50:04,Sem evento,287
2246,SANTOS BRASIL PARTICIPACOES LTDA,2021-07-06 18:24:00,152105130140867,313,TCNU1471063,2022-04-20 09:55:06,Sem evento,287
2247,SANTOS BRASIL PARTICIPACOES LTDA,2021-07-14 09:05:00,152105141463000,305,SUDU5961308,2022-05-13 15:54:57,Sem evento,303
2314,SANTOS BRASIL PARTICIPACOES LTDA,2021-07-21 10:00:00,152105154381775,298,TGHU9887953,2022-04-20 09:56:12,Sem evento,272
2371,SANTOS BRASIL PARTICIPACOES LTDA,2021-07-28 15:34:00,152105167943840,291,MSKU8852797,2022-05-16 08:49:26,Sem evento,291
2403,SANTOS BRASIL PARTICIPACOES LTDA,2021-08-06 09:00:00,152105167946008,283,MAEU8378843,Sem evento,Sem evento,0
2437,SANTOS BRASIL PARTICIPACOES LTDA,2021-08-11 11:13:00,152105180015861,277,HLBU2045539,Sem evento,Sem evento,0
2533,SANTOS BRASIL PARTICIPACOES LTDA,2021-08-16 10:00:00,122105193597307,272,TXGU5267343,Sem evento,Sem evento,0
2535,SANTOS BRASIL PARTICIPACOES LTDA,2021-08-26 16:19:00,122105193356539,262,TCKU7696484,Sem evento,Sem evento,0
2546,SANTOS BRASIL PARTICIPACOES LTDA,2021-09-01 12:30:00,022105197966459,256,MRKU4618900,Sem evento,Sem evento,0


In [15]:
paradigma = mostra_recinto(recintos[4])
paradigma

EUDMARCO - PORTO SECO 23


,Ficha,Recinto,Data de Abertura,numeroCEmercante,Dias sem conclusão,Parecer,Contêiner,Saneamento recebido,TG recebido,fase,Duração
21,2085,EUDMARCO - PORTO SECO,2021-06-14 16:16:00,152105100814240,335,,TGHU6092491,2021-08-13 17:31:11,2021-09-27 10:40:56,3,60
26,2244,EUDMARCO - PORTO SECO,2021-07-07 09:00:00,152105130137564,313,--,XCLU6770681,Sem evento,Sem evento,4,0
27,2248,EUDMARCO - PORTO SECO,2021-07-11 13:43:00,152105141463697,308,"Apreensão total. PEC Enviada em Julho, AR devo...",TEMU6937766,2022-04-28 10:15:35,2021-10-18 10:30:55,3,290
29,2249,EUDMARCO - PORTO SECO,2021-07-16 09:00:00,152105144870773,304,desbloqueio usados,MSMU7099782,Sem evento,2022-02-03 17:47:47,3,202
25,2250,EUDMARCO - PORTO SECO,2021-07-06 17:06:00,152105118504560,313,Desbloqueio dos usados,TCLU9853337,Sem evento,2021-11-04 12:00:51,1,120
22,2255,EUDMARCO - PORTO SECO,2021-06-23 09:06:00,152105117198210,326,--,UACU5173520,Sem evento,2021-11-18 16:59:14,3,148
24,2266,EUDMARCO - PORTO SECO,2021-07-01 09:00:00,152105116021007,319,Desbloqueio dos usados,MRKU5241067,2021-08-23 10:35:09,2022-03-25 19:33:41,3,53
23,2280,EUDMARCO - PORTO SECO,2021-06-23 10:13:00,152105091324109,326,--,UACU5308164,Sem evento,2021-09-30 14:54:45,3,99
28,2286,EUDMARCO - PORTO SECO,2021-07-13 11:28:39,152105141460833,306,Termo de constatações e encaminhamento para DIDAD,HASU4145312,Sem evento,Sem evento,3,0
30,2317,EUDMARCO - PORTO SECO,2021-07-23 09:00:00,152105156840114,297,desbloqueio dos usados,GESU6248806,2021-10-27 11:27:23,2022-05-13 11:29:33,1,96


In [16]:
paradigma[paradigma['Duração']==0]

,Ficha,Recinto,Data de Abertura,numeroCEmercante,Dias sem conclusão,Parecer,Contêiner,Saneamento recebido,TG recebido,fase,Duração
26,2244,EUDMARCO - PORTO SECO,2021-07-07 09:00:00,152105130137564,313,--,XCLU6770681,Sem evento,Sem evento,4,0
28,2286,EUDMARCO - PORTO SECO,2021-07-13 11:28:39,152105141460833,306,Termo de constatações e encaminhamento para DIDAD,HASU4145312,Sem evento,Sem evento,3,0
34,2480,EUDMARCO - PORTO SECO,2021-08-17 17:05:00,152105186235103,271,--,PONU8173369,Sem evento,Sem evento,4,0
43,2975,EUDMARCO - PORTO SECO,2021-11-30 09:00:00,152105274500372,167,,,Sem evento,Sem evento,1,0


In [17]:
paradigma_conclusos = paradigma[paradigma['Duração']!=0]
print('Duração média: ', paradigma_conclusos['Duração'].mean())
print('Quantidade concluídos:', len(paradigma_conclusos))
print('Data primeiro e último:', min(paradigma_conclusos['Data de Abertura']), max(paradigma_conclusos['Data de Abertura']))

Duração média:  163.1578947368421
Quantidade concluídos: 19
Data primeiro e último: 2021-06-14 16:16:00 2021-10-13 11:32:00


In [18]:
resumo_recintos = {}
for recinto in recintos:
    df_recinto = df_pendente_outlet[(df_pendente_outlet.Recinto == recinto) & (df_pendente_outlet.fase !=4)]
    concluidos = 0
    pendentes = 0
    tempoc = 0
    tempop = 0
    for row in df_recinto.iterrows():
        if row[1]['Duração'] == 0:  # Não concluído / pendente
            p = row[1]['Dias sem conclusão']
            tempop += p
            pendentes += 1
        else:  # Concluído
            c = row[1]['Duração']
            tempoc += c
            concluidos += 1
    resumo_recintos[recinto] = {
        'concluídos': concluidos,
        'duração média': 0 if concluidos == 0 else int(tempoc / concluidos),
        'pendentes': pendentes,
        'dias sem conclusão média': 0 if pendentes == 0 else int(tempop / pendentes)
    }
df_resumo = pd.DataFrame.from_dict(resumo_recintos, orient='index')
df_resumo['% Concluído'] = (df_resumo['concluídos'] / (df_resumo['concluídos'] + df_resumo['pendentes']))*100


In [19]:
pd.options.display.float_format = '{:,.2f}%'.format
df_resumo.dropna()

,concluídos,duração média,pendentes,dias sem conclusão média,% Concluído
BANDEIRANTES DEICMAR - CLIA ALEMOA,3,240,4,285,42.86%
BANDEIRANTES DEICMAR - IPA OUTEIRINHOS,3,204,9,293,25.00%
Brasil Terminal Portuario,1,195,0,0,100.00%
EUDMARCO - PORTO SECO,19,163,2,236,90.48%
SANTOS BRASIL LOGISTICA S/A -CLIA GUARUJA,0,0,1,192,0.00%
SANTOS BRASIL LOGISTICA S/A -CLIA SANTOS,1,264,0,0,100.00%
SANTOS BRASIL PARTICIPACOES LTDA,8,253,6,262,57.14%
TRANSBRASA IPA,2,152,0,0,100.00%
